# **```Supervised Neural Machine Translation```**

This notebook runs Seq2Seq model to translate Sanskrit to English, which we translate to Hindi using Google Translate.  

NOTE: This requires GPU to run. With only CPU, processing can be painfully slow, with GPU also, training takes around 1.5 hrs

In [2]:
!rm -rf sample_data
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This is the repo which contains the parallel data we are going to use.

In [ ]:
!unzip /content/drive/MyDrive/sa_hi_zst1_25gb-20240417T133941Z-001.zip -d /content/drive/MyDrive/

Archive:  /content/drive/MyDrive/sa_hi_zst1_25gb-20240417T133941Z-001.zip
   creating: /content/drive/MyDrive/sa_hi_zst1_25gb/.ipynb_checkpoints/
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/contarget.train  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train_source1.txt  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/run.vocab.src  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/en-sa.sa.bped  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train_english  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train_source_n  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train.hi-en.en  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train_hindi  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/train_target_n  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/en-sa.en.all  
  inflating: /content/drive/MyDrive/sa_hi_zst1_25gb/data/valid_source_n  
  inflating: /content/drive/MyDrive/sa_hi_zs

In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/priyanshu2103/Sanskrit-Hindi-Machine-Translation.git

/content/drive/MyDrive
Cloning into 'Sanskrit-Hindi-Machine-Translation'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 124 (delta 46), reused 58 (delta 12), pack-reused 0
Receiving objects: 100% (124/124), 9.25 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
pwd

'/content'

In [3]:
#%cd /content/drive/MyDrive/sa_hi_zst1_25gb/data/
sour_lines = []
tar_lines = []
# with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/train_source1.txt-filtered.sa', 'r') as f:
#   sour_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/train_source2.txt-filtered.en', 'r') as f:
  sour_lines.extend([x.replace('\n', '') for x in f.readlines()])

#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/src-test.txt-filtered.sa', 'r') as f:
  #sour_lines.extend([x.replace('\n', '') for x in f.readlines()])

# with open('manu_english.txt', 'r') as f:
#   eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

# with open('ramayan_english.txt', 'r') as f:
#   eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

# with open('rigveda_english.txt', 'r') as f:
#   eng_lines.extend([x.replace('\n', '') for x in f.readlines()])



# with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/train_target1.txt-filtered.en', 'r') as f:
#   tar_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/train_target2.txt-filtered.hi', 'r') as f:
  tar_lines.extend([x.replace('\n', '') for x in f.readlines()])


#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/tgt-test.txt-filtered.hi', 'r') as f:
  #tar_lines.extend([x.replace('\n', '') for x in f.readlines()])
# with open('manu_sanskrit.txt', 'r') as f:
#   sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

# with open('ramayan_sanskrit.txt', 'r') as f:
#   sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

# with open('rigveda_sanskrit.txt', 'r') as f:
#   sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

print(sour_lines[:100])
print(tar_lines[:100])

print(len(sour_lines))
print(len(tar_lines))

['hi vertical layout', 'hi search mailbox', 'hi select a predefined set of imap headers to fetch note larger sets of headers take longer to download', 'hi fl ush outbox', 'hi autogen template used for the header file', 'hi delete junk messages', 'hi no reason was specified', 'hi contains value', 'hi error during filters loading s filters may not be correct', 'hi delete processed', 'hi the uri of the directory last used to open or save a document', 'hi enable the immed flag see cdrecord manual', 'hi should brasero filter broken symbolic links', 'hi please select an evolution archive to restore', 'hi configure email accounts', 'hi create a shared new memo', 'hi unable to add message to summary unknown reason', 'hi deprecated proxy authentication user name', 'hi in process', 'hi are you sure you want to remove s from the addressbook', 'hi could not open trash s', 'hi the user name e g jim', 'hi move selected headers up one row', 'hi create a new account on the server', 'hi switch to the n

In [4]:
# Randomly shuffling the data into training and dev set, the test data is already provided on github
import random
c = list(zip(sour_lines, tar_lines))
random.shuffle(c)

sour_lines, tar_lines = zip(*c)

train_text_so = sour_lines[:-1374]
train_text_ta = tar_lines[:-1374]

dev_text_so = sour_lines[-1374:]
dev_text_ta = tar_lines[-1374:]

In [5]:
print(len(train_text_so))
print(len(dev_text_so))

12652
1374


In [6]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [7]:
# Reading the data and forming pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Forming pairs of sentences
    pairs = [[train_text_so[i], train_text_ta[i]] for i in range(len(train_text_so))]
    #pairs_v=
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Filtering the data by only keeping sentences of length less than 100 words




In [8]:
MAX_LENGTH = 100

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('source', 'target', False)
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 12652 sentence pairs
Trimmed to 12652 sentence pairs
Counting words...
Counted words:
source 4628
target 6306
['hi anjuta manual', 'anjuta स्थिति']


Seq2Seq Model
=================

A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.


The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Attention Decoder

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

In [12]:
import random
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(random.choice(list(lang.word2index.values())))
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.




In [13]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [16]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there.

In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Google translate API to translate English to Hindi

In [18]:
def evaluate_test(encoder, decoder):
  test = []
  # with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/src-test.txt-filtered.sa', 'r') as f:
  #with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/test_sanskrit.sa', 'r') as f:
    #test_text = [x.replace('\n', '') for x in f.readlines()]
  #for sa_text in test_text:
  for sa_text in dev_text_so:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    #print('<', output_sentence)
    # print('')
    test.append(output_sentence)
    #print(test)

  # translated = []
  # print(len(test))

  # #for line in test:
  # ans = translator.translate(test, dest='hi',src='auto')
  # print(ans)

  #translated.append(ans.text)

  with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/translated_hi_en.hi', 'w') as f:
    for line in test:
      f.write(line + '\n')

  # calculate bleu score
  import nltk
  #with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/tgt-test.txt-filtered.hi', 'r') as f: # ground truth file
    #reference = [x.replace('\n', '') for x in f.readlines()]
  reference = dev_text_ta

  with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/translated_hi_en.hi', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

  from nltk.translate.bleu_score import SmoothingFunction
  smoothie = SmoothingFunction().method4
  #print(test_text,reference,canditate)
  #print("bleu", nltk.translate.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie))
  print("bleu", nltk.translate.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie))
  print("bleu1", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(1,0,0,0), smoothing_function=smoothie))
  print("bleu2", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(0,1,0,0), smoothing_function=smoothie))
  print("bleu3", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(0,0,1,0), smoothing_function=smoothie))
  print("bleu4", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(1,0,0,1), smoothing_function=smoothie))
  print("chrf",nltk.translate.chrf_score.corpus_chrf(reference, candidate))

In [19]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 80000, print_every=5000)

2m 18s (- 34m 31s) (5000 6%) 5.0264
4m 16s (- 29m 56s) (10000 12%) 4.6088
6m 13s (- 26m 56s) (15000 18%) 4.1439
8m 7s (- 24m 23s) (20000 25%) 3.7403
10m 4s (- 22m 10s) (25000 31%) 3.4642
12m 0s (- 20m 1s) (30000 37%) 3.2426
13m 56s (- 17m 55s) (35000 43%) 2.9818
15m 56s (- 15m 56s) (40000 50%) 2.8306
17m 54s (- 13m 55s) (45000 56%) 2.6123
19m 51s (- 11m 54s) (50000 62%) 2.4954
21m 50s (- 9m 55s) (55000 68%) 2.3220
23m 50s (- 7m 56s) (60000 75%) 2.2539
26m 4s (- 6m 1s) (65000 81%) 2.1195
28m 10s (- 4m 1s) (70000 87%) 2.0049
30m 18s (- 2m 1s) (75000 93%) 1.9486
32m 33s (- 0m 0s) (80000 100%) 1.8421


In [20]:
# saving the trained models
torch.save(encoder1.state_dict(), '/content/drive/MyDrive/sa_hi_zst1_25gb/trained_encoder_enhi.pth')
torch.save(attn_decoder1.state_dict(), '/content/drive/MyDrive/sa_hi_zst1_25gb/trained_decoder_enhi.pth')

In [21]:
evaluate_test(encoder1, attn_decoder1)

bleu 0.00016277012748616796
bleu1 0.32518520196642575
bleu2 2.5007936808151465e-05
bleu3 1.2919025360256727e-05
bleu4 2.172657156946199e-06
chrf 0.30251590806291334


In [36]:
test_text = dev_text_so
reference = dev_text_ta

with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/translated_hi_en.hi', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
for i in range(len(test_text)):
  #print("source english:",test_text[i])
  print("actual hindi:",reference[i])
  print("predicted hindi:",candidate[i])

actual hindi: रेखांकित स्तंभ
predicted hindi: स्तंभ स्तंभ <EOS>
actual hindi: सभी ढेर भरने लायक पत्ते होते तक पहले जैसा करें
predicted hindi: सभी को को को को को को को को को को को को को है <EOS>
actual hindi: नाम के लिए ढूँढें
predicted hindi: खोज के लिए खोजें <EOS>
actual hindi: webdav propfind परिणाम में कोई अनुक्रिया बॉडी नहीं
predicted hindi: गनोम अनुक्रिया जब की <EOS>
actual hindi: immed फ्लैग के साथ cdrecord के साथ प्रयुक्त
predicted hindi: immed के के के के के साथ साथ साथ साथ <EOS>
actual hindi: अधिसूचना क्षेत्र में आने वाली घटनाओं प्रदर्शित करें
predicted hindi: अधिसूचना क्षेत्र में प्रतीक क्षेत्र दिखाएँ <EOS>
actual hindi: नई ऑडियो परियोजना a
predicted hindi: ऑडियो ऑडियो परियोजना <EOS>
actual hindi: पंचांग संदेश
predicted hindi: पंचांग संदेश <EOS>
actual hindi: दीर्घा के अंतिम छवि पर जाएँ
predicted hindi: दीर्घा के अंतिम छवि <EOS>
actual hindi: लागू करने के लिए टिंट
predicted hindi: लागू लागू के लिए लागू <EOS>
actual hindi: s पहले तर्क time t होने की आशा करता
predicted hindi: ए

In [ ]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [ ]:
#encoder1 = TheModelClass(*args, **kwargs)
encoder1.load_state_dict(torch.load('/content/drive/MyDrive/sa_hi_zst1_25gb/trained_encoder.pth'))
#attn_decoder1=TheModelClass(*args, **kwargs)
attn_decoder1.load_state_dict(torch.load('/content/drive/MyDrive/sa_hi_zst1_25gb/trained_decoder.pth'))
#model.eval()

In [ ]:
trainIters(encoder1, attn_decoder1, 40000, print_every=5000)

3m 47s (- 26m 29s) (5000 12%) 4.2529
7m 30s (- 22m 30s) (10000 25%) 4.2561
11m 21s (- 18m 56s) (15000 37%) 4.1935
15m 3s (- 15m 3s) (20000 50%) 4.1411
18m 45s (- 11m 15s) (25000 62%) 4.1239
22m 24s (- 7m 28s) (30000 75%) 4.0948
26m 6s (- 3m 43s) (35000 87%) 4.1253
29m 46s (- 0m 0s) (40000 100%) 4.0395


In [ ]:
# saving the trained models
torch.save(encoder1.state_dict(), '/content/drive/MyDrive/sa_hi_zst1_25gb/trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), '/content/drive/MyDrive/sa_hi_zst1_25gb/trained_decoder.pth')

In [ ]:
### for spearman correlation between hum eval and BLEU1,2,3,4 and chrf####

In [23]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
b1=[]
b2=[]
b3=[]
b4=[]
chRf=[]
#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/src-test.txt-filtered.sa', 'r') as f:
test_text = dev_text_so
reference = dev_text_ta

with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/translated_hi_en.hi', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
for i in range(0,50):
  b1.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(1,0,0,0), smoothing_function=smoothie))
  b2.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,1,0,0), smoothing_function=smoothie))
  b3.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,0,1,0), smoothing_function=smoothie))
  b4.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,0,0,1), smoothing_function=smoothie))
  chRf.append(nltk.translate.chrf_score.sentence_chrf(reference[i], candidate[i]))

In [24]:
print(b1)
print(b2)
print(b3)
print(b4)
print(chRf)

[0.35294117647058826, 0.15686274509803924, 0.3181818181818182, 0.5384615384615384, 0.23809523809523808, 0.4166666666666667, 0.4230769230769231, 0.5555555555555556, 0.64, 0.3333333333333333, 0.2222222222222222, 0.38461538461538464, 0.4117647058823529, 0.2238805970149254, 0.21052631578947367, 0.14925373134328357, 0.20987654320987653, 0.45454545454545453, 0.3541666666666667, 0.34545454545454546, 0.45, 0.4375, 0.42857142857142855, 0.3499999999999999, 0.375, 0.5789473684210527, 0.14285714285714285, 0.4375, 0.2692307692307693, 0.391304347826087, 0.2926829268292683, 0.3793103448275862, 0.5, 0.1590909090909091, 0.16666666666666669, 0.25, 0.38095238095238093, 0.47619047619047616, 0.28, 0.2, 0.36, 0.38235294117647056, 0.59375, 0.3333333333333333, 0.39473684210526316, 0.25, 0.5238095238095238, 0.3499999999999999, 0.3972602739726027, 0.35294117647058826]
[0.017707583400351348, 0.007863651265448652, 0.014719249777896742, 0.013032386152085925, 0.009116267361666756, 0.008236597895548708, 0.0130323861

In [25]:
def rankify(X):

	N = len(X)

	# Rank Vector
	Rank_X = [None for _ in range(N)]

	for i in range(N):

		r = 1
		s = 1

		# Count no of smaller elements
		# in 0 to i-1
		for j in range(i):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Count no of smaller elements
		# in i+1 to N-1
		for j in range(i+1, N):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Use Fractional Rank formula
		# fractional_rank = r + (n-1)/2
		Rank_X[i] = r + (s-1) * 0.5

	# Return Rank Vector
	return Rank_X

In [26]:
rank_y=rankify(chRf)

In [27]:
#printVector(rank_y)
hum4=[]
hum3=[]
hum2=[]
hum1=[]
#print(rank_y)(rank_y) is of chrfscore
for i in range(len(rank_y)):
   if rank_y[i]>=45:
    hum4.append(i)
   elif rank_y[i]>=40 and  rank_y[i]<45:
    hum3.append(i)
   elif rank_y[i]>=25 and rank_y[i]<40:
    hum2.append(i)
   else:
    hum1.append(i)

print(hum1)
print(hum2)
print(hum3)
print(hum4)


[1, 3, 4, 10, 13, 14, 15, 20, 21, 22, 23, 26, 30, 31, 33, 34, 36, 37, 39, 40, 41, 43, 45, 47]
[0, 2, 9, 12, 16, 18, 19, 24, 27, 28, 35, 38, 44, 48, 49]
[5, 11, 17, 29, 46]
[6, 7, 8, 25, 32, 42]


In [28]:
### humrank calculated for 50 sentences from value 1 to 4
humval=[]
for i in hum1:
    humval.insert(i,1)
for i in hum2:
    humval.insert(i,2)
for i in hum3:
    humval.insert(i,3)
for i in hum4:
    humval.insert(i,4)
print(humval)


[2, 1, 2, 1, 1, 3, 4, 4, 4, 1, 1, 1, 1, 2, 3, 1, 1, 2, 1, 1, 3, 1, 2, 1, 2, 4, 2, 1, 1, 1, 1, 2, 4, 1, 3, 1, 2, 2, 1, 1, 1, 1, 4, 1, 2, 2, 2, 2, 2, 3]


In [29]:
humval=[2, 1, 2, 1, 1, 3, 4, 4, 4, 1, 1, 1, 1, 2, 3, 1, 1, 2, 1, 1, 3, 1, 2, 1, 2, 4, 2, 1, 1, 1, 1, 2, 4, 1, 3, 1, 2, 2, 1, 1, 1, 1, 4, 1, 2, 2, 2, 2, 2, 3]


In [30]:
b1=[0.35294117647058826, 0.15686274509803924, 0.3181818181818182, 0.5384615384615384, 0.23809523809523808, 0.4166666666666667, 0.4230769230769231, 0.5555555555555556, 0.64, 0.3333333333333333, 0.2222222222222222, 0.38461538461538464, 0.4117647058823529, 0.2238805970149254, 0.21052631578947367, 0.14925373134328357, 0.20987654320987653, 0.45454545454545453, 0.3541666666666667, 0.34545454545454546, 0.45, 0.4375, 0.42857142857142855, 0.3499999999999999, 0.375, 0.5789473684210527, 0.14285714285714285, 0.4375, 0.2692307692307693, 0.391304347826087, 0.2926829268292683, 0.3793103448275862, 0.5, 0.1590909090909091, 0.16666666666666669, 0.25, 0.38095238095238093, 0.47619047619047616, 0.28, 0.2, 0.36, 0.38235294117647056, 0.59375, 0.3333333333333333, 0.39473684210526316, 0.25, 0.5238095238095238, 0.3499999999999999, 0.3972602739726027, 0.35294117647058826]
b2=[0.017707583400351348, 0.007863651265448652, 0.014719249777896742, 0.013032386152085925, 0.009116267361666756, 0.008236597895548708, 0.013032386152085925, 0.017002186811153907, 0.0134119826036175, 0.012676295638478188, 0.008651505658568908, 0.013032386152085925, 0.017707583400351348, 0.006370746393016614, 0.01635799432870244, 0.006370746393016614, 0.005493061443340548, 0.01092658612958275, 0.008236597895548708, 0.007420987380060135, 0.01576701196607363, 0.018483924814931874, 0.020300440997040446, 0.009458665266958812, 0.011179793234837825, 0.01635799432870244, 0.01522261218861711, 0.011179793234837825, 0.013032386152085925, 0.01425224643604159, 0.00928393016676077, 0.012026056535665977, 0.01234149818583409, 0.008800441009112238, 0.007526384993517495, 0.011179793234837825, 0.01522261218861711, 0.009116267361666756, 0.0134119826036175, 0.019343215722158646, 0.0134119826036175, 0.01068594098368534, 0.011179793234837825, 0.017002186811153907, 0.009831313945206446, 0.011179793234837825, 0.01522261218861711, 0.01576701196607363, 0.005958971446039433, 0.017707583400351348]
b3=[0.009444044480187383, 0.00401206697216768, 0.007727606133395791, 0.006787701120878089, 0.004672087022854211, 0.0042078271857694475, 0.006787701120878089, 0.009032411743425518, 0.00699755614101783, 0.006591673732008659, 0.004426351732291069, 0.006787701120878089, 0.009444044480187383, 0.0032343789379930495, 0.00866011464460718, 0.0032343789379930495, 0.002781296933336987, 0.005639528324945938, 0.0042078271857694475, 0.003780503004936293, 0.008321478537649979, 0.009902102579427786, 0.010996072206730241, 0.004853788755413074, 0.005776226504666211, 0.00866011464460718, 0.008011901151903745, 0.005776226504666211, 0.006787701120878089, 0.0074654624188789254, 0.004760989829108089, 0.006235733018493471, 0.006408085596490774, 0.0045049876594265015, 0.0038355615832348804, 0.005776226504666211, 0.008011901151903745, 0.004672087022854211, 0.00699755614101783, 0.010415577696546965, 0.00699755614101783, 0.005509938319712753, 0.005776226504666211, 0.009032411743425518, 0.005052202999619981, 0.005776226504666211, 0.008011901151903745, 0.008321478537649979, 0.003021450310667881, 0.009444044480187383]
b4=[0.005059309542957528, 0.002047825850377254, 0.004067161122839888, 0.003541409280458132, 0.0023959420630021585, 0.002150667228282162, 0.003541409280458132, 0.004817286263160275, 0.0036578134373502283, 0.0034331634020878417, 0.0022658705296251893, 0.003541409280458132, 0.005059309542957528, 0.0016424580544495955, 0.004600685904947562, 0.0016424580544495955, 0.0014084772931642435, 0.002913756301222066, 0.002150667228282162, 0.001926602492900226, 0.004405488637579399, 0.005331901388922657, 0.005997857567307406, 0.0024924861176445524, 0.0029877033644825227, 0.004600685904947562, 0.004228503385726976, 0.0029877033644825227, 0.003541409280458132, 0.003919367769911436, 0.0024431395175686242, 0.0032377844519100724, 0.0033322045101752034, 0.002307432703608697, 0.001955384336551114, 0.0029877033644825227, 0.004228503385726976, 0.0023959420630021585, 0.0036578134373502283, 0.005641771252296272, 0.0036578134373502283, 0.00284383913275497, 0.0029877033644825227, 0.004817286263160275, 0.002598275828375989, 0.0029877033644825227, 0.004228503385726976, 0.004405488637579399, 0.001532306943267282, 0.005059309542957528]
chRf=[0.2384671848609419, 0.0547110067600026, 0.22879556099756035, 0.19207188649726858, 0.20339079588909292, 0.4515163950181577, 0.7355704586790363, 0.942309639927085, 0.6934873112959384, 0.30170686507356886, 0.1876319264979157, 0.4473876863048311, 0.3217903539897505, 0.11665256840808076, 0.043103448275862155, 0.12684165152660495, 0.3553561035604778, 0.6732583423530748, 0.39431821551293855, 0.36240203540648, 0.14521955754812088, 0.18158462967024244, 0.154032346517192, 0.1342158363253686, 0.2739010115487542, 0.8682656538690955, 0.042735042735042826, 0.27369413383335517, 0.373444402669242, 0.45191445397260477, 0.130553098493673, 0.06258248602547775, 0.6863810564647174, 0.0363575955686218, 0.11562574662572937, 0.2313776148041414, 0.09635268831504751, 0.10346869140031449, 0.24377010823836923, 0.02398081534772192, 0.22280117955014792, 0.1827553847983574, 0.8046825172309168, 0.17177197802197805, 0.32771378955589486, 0.08849147516679029, 0.4867293926117456, 0.15276488531896953, 0.39295957788864283, 0.2943225333318212]

In [31]:
# Python3 Program to find correlation coefficient


# Utility Function to print
# a Vector
def printVector(X):
	print(*X)

# Function returns the rank vector
# of the set of observations


def rankify(X):

	N = len(X)

	# Rank Vector
	Rank_X = [None for _ in range(N)]

	for i in range(N):

		r = 1
		s = 1

		# Count no of smaller elements
		# in 0 to i-1
		for j in range(i):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Count no of smaller elements
		# in i+1 to N-1
		for j in range(i+1, N):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Use Fractional Rank formula
		# fractional_rank = r + (n-1)/2
		Rank_X[i] = r + (s-1) * 0.5

	# Return Rank Vector
	return Rank_X


# function that returns
# Pearson correlation coefficient.
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b1
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))


# This code is contributed by phasing17


Vector X
0.35294117647058826 0.15686274509803924 0.3181818181818182 0.5384615384615384 0.23809523809523808 0.4166666666666667 0.4230769230769231 0.5555555555555556 0.64 0.3333333333333333 0.2222222222222222 0.38461538461538464 0.4117647058823529 0.2238805970149254 0.21052631578947367 0.14925373134328357 0.20987654320987653 0.45454545454545453 0.3541666666666667 0.34545454545454546 0.45 0.4375 0.42857142857142855 0.3499999999999999 0.375 0.5789473684210527 0.14285714285714285 0.4375 0.2692307692307693 0.391304347826087 0.2926829268292683 0.3793103448275862 0.5 0.1590909090909091 0.16666666666666669 0.25 0.38095238095238093 0.47619047619047616 0.28 0.2 0.36 0.38235294117647056 0.59375 0.3333333333333333 0.39473684210526316 0.25 0.5238095238095238 0.3499999999999999 0.3972602739726027 0.35294117647058826
Rankings of X
23.5 3.0 17.0 46.0 11.0 36.0 37.0 47.0 50.0 18.5 9.0 31.0 35.0 10.0 8.0 2.0 7.0 42.0 25.0 20.0 41.0 39.5 38.0 21.5 27.0 48.0 1.0 39.5 14.0 32.0 16.0 28.0 44.0 4.0 5.0 12.5 2

In [32]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b2
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.017707583400351348 0.007863651265448652 0.014719249777896742 0.013032386152085925 0.009116267361666756 0.008236597895548708 0.013032386152085925 0.017002186811153907 0.0134119826036175 0.012676295638478188 0.008651505658568908 0.013032386152085925 0.017707583400351348 0.006370746393016614 0.01635799432870244 0.006370746393016614 0.005493061443340548 0.01092658612958275 0.008236597895548708 0.007420987380060135 0.01576701196607363 0.018483924814931874 0.020300440997040446 0.009458665266958812 0.011179793234837825 0.01635799432870244 0.01522261218861711 0.011179793234837825 0.013032386152085925 0.01425224643604159 0.00928393016676077 0.012026056535665977 0.01234149818583409 0.008800441009112238 0.007526384993517495 0.011179793234837825 0.01522261218861711 0.009116267361666756 0.0134119826036175 0.019343215722158646 0.0134119826036175 0.01068594098368534 0.011179793234837825 0.017002186811153907 0.009831313945206446 0.011179793234837825 0.01522261218861711 0.01576701196607363 0

In [33]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b3
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.009444044480187383 0.00401206697216768 0.007727606133395791 0.006787701120878089 0.004672087022854211 0.0042078271857694475 0.006787701120878089 0.009032411743425518 0.00699755614101783 0.006591673732008659 0.004426351732291069 0.006787701120878089 0.009444044480187383 0.0032343789379930495 0.00866011464460718 0.0032343789379930495 0.002781296933336987 0.005639528324945938 0.0042078271857694475 0.003780503004936293 0.008321478537649979 0.009902102579427786 0.010996072206730241 0.004853788755413074 0.005776226504666211 0.00866011464460718 0.008011901151903745 0.005776226504666211 0.006787701120878089 0.0074654624188789254 0.004760989829108089 0.006235733018493471 0.006408085596490774 0.0045049876594265015 0.0038355615832348804 0.005776226504666211 0.008011901151903745 0.004672087022854211 0.00699755614101783 0.010415577696546965 0.00699755614101783 0.005509938319712753 0.005776226504666211 0.009032411743425518 0.005052202999619981 0.005776226504666211 0.008011901151903745 0.0

In [34]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b4
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.005059309542957528 0.002047825850377254 0.004067161122839888 0.003541409280458132 0.0023959420630021585 0.002150667228282162 0.003541409280458132 0.004817286263160275 0.0036578134373502283 0.0034331634020878417 0.0022658705296251893 0.003541409280458132 0.005059309542957528 0.0016424580544495955 0.004600685904947562 0.0016424580544495955 0.0014084772931642435 0.002913756301222066 0.002150667228282162 0.001926602492900226 0.004405488637579399 0.005331901388922657 0.005997857567307406 0.0024924861176445524 0.0029877033644825227 0.004600685904947562 0.004228503385726976 0.0029877033644825227 0.003541409280458132 0.003919367769911436 0.0024431395175686242 0.0032377844519100724 0.0033322045101752034 0.002307432703608697 0.001955384336551114 0.0029877033644825227 0.004228503385726976 0.0023959420630021585 0.0036578134373502283 0.005641771252296272 0.0036578134373502283 0.00284383913275497 0.0029877033644825227 0.004817286263160275 0.002598275828375989 0.0029877033644825227 0.00422

In [35]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = chRf
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.2384671848609419 0.0547110067600026 0.22879556099756035 0.19207188649726858 0.20339079588909292 0.4515163950181577 0.7355704586790363 0.942309639927085 0.6934873112959384 0.30170686507356886 0.1876319264979157 0.4473876863048311 0.3217903539897505 0.11665256840808076 0.043103448275862155 0.12684165152660495 0.3553561035604778 0.6732583423530748 0.39431821551293855 0.36240203540648 0.14521955754812088 0.18158462967024244 0.154032346517192 0.1342158363253686 0.2739010115487542 0.8682656538690955 0.042735042735042826 0.27369413383335517 0.373444402669242 0.45191445397260477 0.130553098493673 0.06258248602547775 0.6863810564647174 0.0363575955686218 0.11562574662572937 0.2313776148041414 0.09635268831504751 0.10346869140031449 0.24377010823836923 0.02398081534772192 0.22280117955014792 0.1827553847983574 0.8046825172309168 0.17177197802197805 0.32771378955589486 0.08849147516679029 0.4867293926117456 0.15276488531896953 0.39295957788864283 0.2943225333318212
Rankings of X
27.0 5